In [71]:
import pandas as pd
import numpy as np
import re
import os
import datetime as dt

In [62]:
df_regal = pd.read_excel("General Claim Returns 51-motors.xlsx")
df_supp  = pd.read_excel("Regal_shipment_number.xlsx")

In [67]:
def rep(x):
    if x.startswith('51-102497-'):
        return '51-101920-00'
    elif x.startswith('51-103823-'):
        return '51-101984-00'
    elif x.startswith('51-104337-'):
        return '51-101879-00'
    elif x.startswith('51-104338-'):
        return '51-101880-00'
    elif x.startswith('51-104339-'):
        return '51-101920-00'
    elif x.startswith('51-104340-'):
        return '51-101984-00'
    else:
        return x
    
df_new = df_regal.copy() 
df_regal['Entered Failed Part'] = df_regal['Entered Failed Part'].apply(rep)
df_regal[df_regal['Entered Failed Part'].isin(['51-101920-00','51-101984-00','51-101879-00','51-101880-00','51-101920-00','51-101984-00'])]['Entered Failed Part'].value_counts()

51-101920-00    20537
51-101880-00    19310
51-101879-00     4239
51-101984-00     2553
Name: Entered Failed Part, dtype: int64

In [64]:
df_regal['part_family']= df_regal['Entered Failed Part'].apply(lambda x: "-".join(x.split('-',2)[:2]))
df_regal['part_family'] = df_regal['part_family'].replace({'51-102497':'51-101920' , '51-103823': '51-101984',\
                                                      '51-104337':'51-101879','51-104338':'51-101880',\
                                                        '51-104339':'51-101920','51-104340':'51-101984'})

df_regal[df_regal.part_family.isin(['51-101920','51-101984','51-101879','51-101880'])]['part_family'].value_counts()

51-101880    32694
51-101920    25933
51-101879     7095
51-101984     3488
Name: part_family, dtype: int64

In [76]:
df_regal["num_days"] = (pd.to_datetime(df_regal['Claim Transaction Date'])-pd.to_datetime(df_regal['Failed Unit Production Date'])).dt.days

In [77]:
df_regal.columns

Index(['Claim Number', 'Failed Unit Product Class',
       'Failed Unit Customer Name', 'Installer Name',
       'Entered Failed Unit Number', 'Entered Failed Unit Serial',
       'Entered Repl Unit', 'Entered Repl Unit Serial', 'Entered Failed Part',
       'Entered Failed Part Serial Number', 'Entered Repl Part',
       'Claim Transaction Date', 'TRANSACTION YEAR',
       'Failed Unit Production Date', 'Claim Type Name', 'QA Reason Code Name',
       'QA Reason Code Comments', 'Price', 'Labor Amount', 'Reserve Cost',
       'Failed Part Product Name', 'DIFFERENCE', 'part_family', 'num_days'],
      dtype='object')

In [87]:
df_new = df_regal.copy()
df_reg_filt = df_new[df_new['DIFFERENCE']=="KEEP"]
df_reg_filt.to_excel("FILTERED_REGAL.xlsx",index=False)

In [91]:
df_reg_filt['TRANSACTION YEAR'].sort_values().unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int64)

In [138]:
df_reg_filt['num_claims'] = 1
df_reg_filt['Production_year'] = pd.to_datetime(df_reg_filt['Failed Unit Production Date']).dt.year


C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_3768\2323969533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg_filt['num_claims'] = 1
C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_3768\2323969533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg_filt['Production_year'] = pd.to_datetime(df_reg_filt['Failed Unit Production Date']).dt.year


In [194]:
df_reg_filt.columns

Index(['Claim Number', 'Failed Unit Product Class',
       'Failed Unit Customer Name', 'Installer Name',
       'Entered Failed Unit Number', 'Entered Failed Unit Serial',
       'Entered Repl Unit', 'Entered Repl Unit Serial', 'Entered Failed Part',
       'Entered Failed Part Serial Number', 'Entered Repl Part',
       'Claim Transaction Date', 'TRANSACTION YEAR',
       'Failed Unit Production Date', 'Claim Type Name', 'QA Reason Code Name',
       'QA Reason Code Comments', 'Price', 'Labor Amount', 'Reserve Cost',
       'Failed Part Product Name', 'DIFFERENCE', 'part_family', 'num_days',
       'num_claims', 'Production_year'],
      dtype='object')

In [195]:
df_reg_grp = df_reg_filt.groupby(['Failed Unit Product Class','TRANSACTION YEAR','Entered Failed Part'])[['num_claims','Reserve Cost']].sum().reset_index()
df_reg_grp = df_reg_grp.rename(columns={'TRANSACTION YEAR':'Year'})
df_reg_grp.head()

,Year,Entered Failed Part,num_claims,Reserve Cost
0,2015,51-100837-04,3,140.55825
1,2015,51-100998-05,10,306.78250
2,2015,51-100998-08,11,405.78725
3,2015,51-100998-12,2,74.45600
4,2015,51-100998-13,37,1235.21725


In [196]:
df_supp
# Unpivot the DataFrame
df_unpivoted = pd.melt(df_supp, id_vars=['Part Number'], var_name='Year', value_name='Value')
 # Drop rows where 'Value' is None 
df_unpivoted = df_unpivoted.dropna(subset=['Value']) 
df_unpivoted =df_unpivoted.rename(columns= {'Part Number':'Entered Failed Part','Value':'shipped_units'})
# df_unpivoted['Value'] = df_unpivoted['Value'].round(decimals=0).astype(object)
df_unpivoted

,Entered Failed Part,Year,shipped_units
0,51-100837-01,2015,25.0
1,51-100837-02,2015,25.0
2,51-100837-04,2015,6904.0
3,51-100837-05,2015,99.0
4,51-100837-07,2015,315.0
...,...,...,...
2754,51-42534-39,2023,250.0
2755,51-42534-40,2023,742.0
2756,51-42534-41,2023,269.0
2757,51-42534-42,2023,15.0


In [184]:
df_reg_grp[df_reg_grp['Entered Failed Part']=='51-100998-24']

,Year,Entered Failed Part,num_claims,Reserve Cost
7,2015,51-100998-24,3,108.14775
193,2016,51-100998-24,18,648.88650
495,2017,51-100998-24,35,1070.66375
849,2018,51-100998-24,111,3599.01075
1227,2019,51-100998-24,268,9020.50160
1674,2020,51-100998-24,354,12112.52185
2138,2021,51-100998-24,460,15796.15565
2629,2022,51-100998-24,466,19249.05675
3112,2023,51-100998-24,459,19982.85575
3719,2024,51-100998-24,50,2156.88000


In [197]:
df_final_grped = df_unpivoted.merge(df_reg_grp,on=['Year','Entered Failed Part'],how='left')
df_final_grped[df_final_grped['Entered Failed Part']=='51-100998-24']
df_final_grped.to_excel('Final_data.xlsx',index=False)

In [198]:
df_final_grped['Rolling_Avg_Shipped_Units'] = df_final_grped.groupby('Entered Failed Part')['shipped_units'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True) 
# Display the DataFrame with rolling average 
df_final_grped[['Entered Failed Part', 'Year', 'shipped_units', 'num_claims', 'Rolling_Avg_Shipped_Units']]
df_final_grped['Rolling_Avg_Shipped_Units']  = df_final_grped['Rolling_Avg_Shipped_Units'].round(decimals=0)
df_final_grped['Reserve Cost']  = df_final_grped['Reserve Cost'].round(decimals=2)

df_final_grped[df_final_grped['Entered Failed Part']=='51-100998-24']


,Entered Failed Part,Year,shipped_units,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units
18,51-100998-24,2015,6725.0,3.0,108.15,6725.0
192,51-100998-24,2016,32473.0,18.0,648.89,19599.0
371,51-100998-24,2017,39948.0,35.0,1070.66,26382.0
550,51-100998-24,2018,43524.0,111.0,3599.01,30668.0
724,51-100998-24,2019,40465.0,268.0,9020.50,32627.0
904,51-100998-24,2020,61582.0,354.0,12112.52,37453.0
1081,51-100998-24,2021,54268.0,460.0,15796.16,45377.0
1261,51-100998-24,2022,53983.0,466.0,19249.06,48962.0
1448,51-100998-24,2023,40793.0,459.0,19982.86,49102.0


In [199]:
df_final_grped['rate'] = round((df_final_grped['num_claims']/df_final_grped['Rolling_Avg_Shipped_Units'])*100,2)
df_final_grped

,Entered Failed Part,Year,shipped_units,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units,rate
0,51-100837-01,2015,25.0,NaN,NaN,25.0,NaN
1,51-100837-02,2015,25.0,NaN,NaN,25.0,NaN
2,51-100837-04,2015,6904.0,3.0,140.56,6904.0,0.04
3,51-100837-05,2015,99.0,NaN,NaN,99.0,NaN
4,51-100837-07,2015,315.0,NaN,NaN,315.0,NaN
...,...,...,...,...,...,...,...
1600,51-42534-39,2023,250.0,NaN,NaN,200.0,NaN
1601,51-42534-40,2023,742.0,13.0,1049.20,419.0,3.10
1602,51-42534-41,2023,269.0,4.0,344.79,136.0,2.94
1603,51-42534-42,2023,15.0,NaN,NaN,6.0,NaN


In [200]:
df_final_grped.to_excel("SCORE_CARD_REGAL.xlsx",index=False)

### BY PRODUCT FAMILY

In [247]:
# df_regal[df_regal['TRANSACTION YEAR']==2023].groupby('part_family')
df_regal['Claims_count'] =1
df_2023_regal = df_regal[df_regal['TRANSACTION YEAR']==2023].groupby(['TRANSACTION YEAR','part_family'])[['Claims_count','Reserve Cost']].sum().reset_index()
df_2023_regal = df_2023_regal.rename(columns={'Reserve Cost':'Reserve_cost_2023','TRANSACTION YEAR':'Year'})
df_2023_regal.head()

,Year,part_family,Claims_count,Reserve_cost_2023
0,2023,51-100837,522,28919.557000
1,2023,51-100998,1528,77287.530400
2,2023,51-100999,73,3115.615250
3,2023,51-101728,1727,99426.806300
4,2023,51-101774,4506,183169.009947


In [233]:
df_reg_grp_fam = df_reg_filt.groupby(['TRANSACTION YEAR','part_family'])[['num_claims','Reserve Cost']].sum().reset_index()
df_reg_grp_fam = df_reg_grp_fam.rename(columns={'TRANSACTION YEAR':'Year'})
df_supp
df_reg_grp_fam
# Unpivot the DataFrame
df_unpivoted_fam = pd.melt(df_supp, id_vars=['Part Number'], var_name='Year', value_name='Value')
 # Drop rows where 'Value' is None 
df_unpivoted_fam = df_unpivoted_fam.dropna(subset=['Value']) 
df_unpivoted_fam =df_unpivoted_fam.rename(columns= {'Part Number':'Entered Failed Part','Value':'shipped_units'})
df_unpivoted_fam['part_family'] =df_unpivoted_fam['Entered Failed Part'].apply(lambda x: "-".join(x.split('-',2)[:2]))
df_unpivoted_family = df_unpivoted_fam.groupby(['part_family','Year'])['shipped_units'].sum().reset_index()
df_final_grped_family = df_unpivoted_family.merge(df_reg_grp_fam,on=['Year','part_family'],how='left')
# df_final_grped_family[df_final_grped_family['Entered Failed Part']=='51-100998-24']
df_final_grped_family.to_excel('Final_data_family.xlsx',index=False)
df_final_grped_family['Rolling_Avg_Shipped_Units'] = df_final_grped_family.groupby('part_family')['shipped_units'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True) 
# Display the DataFrame with rolling average 
df_final_grped_family[['part_family', 'Year', 'shipped_units', 'num_claims', 'Rolling_Avg_Shipped_Units']]
df_final_grped_family['Rolling_Avg_Shipped_Units']  = df_final_grped_family['Rolling_Avg_Shipped_Units'].round(decimals=0)
df_final_grped_family['Reserve Cost']  = df_final_grped_family['Reserve Cost'].round(decimals=2)
df_final_grped_family['rate'] = round((df_final_grped_family['num_claims']/df_final_grped_family['Rolling_Avg_Shipped_Units'])*100,2)
# df_final_grped['rate'] = round((df_final_grped['num_claims']/df_final_grped['Rolling_Avg_Shipped_Units'])*100,2)
df_final_grped_family.to_excel("SCORE_CARD_REGAL_FAMILY.xlsx",index=False)

df_final_grped_family[df_final_grped_family['part_family'].str.startswith('51-100998')]

# df_unpivoted
# df_reg_grp.head()


,part_family,Year,shipped_units,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units,rate
9,51-100998,2015,39454.0,83.0,3000.34,39454.0,0.21
10,51-100998,2016,66706.0,329.0,11664.53,53080.0,0.62
11,51-100998,2017,73399.0,391.0,14454.02,59853.0,0.65
12,51-100998,2018,75684.0,509.0,20326.71,63811.0,0.80
13,51-100998,2019,69037.0,774.0,31405.87,64856.0,1.19
14,51-100998,2020,92601.0,883.0,35005.56,69480.0,1.27
15,51-100998,2021,88433.0,1063.0,44323.70,77643.0,1.37
16,51-100998,2022,89048.0,1182.0,62634.51,81367.0,1.45
17,51-100998,2023,80397.0,1147.0,58060.67,82533.0,1.39


In [249]:
df_final_grped_family.merge(df_2023_regal,on=['part_family','Year'],how='left').to_excel('SCORE_CARD_REGAL_FAMILY_UPDATED.xlsx',index=False)
